In [1]:
!pip install langchain langchain-community chromadb ibm-watsonx-ai gradio

  Using cached langchain-0.3.26-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached chromadb-1.0.15-cp39-abi3-macosx_11_0_arm64.whl.metadata (7.0 kB)
  Using cached langchain_core-0.3.68-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
  Using cached langsmith-0.4.4-py3-none-any.whl.metadata (15 kB)
  Using cached sqlalchemy-2.0.41-cp313-cp313-macosx_11_0_arm64.whl.metadata (9.6 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached aiohttp-3.12.13-cp313-cp313-macosx_11_0_arm64.whl.metadata (7.6 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.10.1-py3-no

In [3]:
!pip install langchain_ibm

  Using cached langchain_ibm-0.3.14-py3-none-any.whl.metadata (5.2 kB)
  Using cached ibm_watsonx_ai-1.3.28-py3-none-any.whl.metadata (6.8 kB)
Using cached langchain_ibm-0.3.14-py3-none-any.whl (29 kB)
Using cached ibm_watsonx_ai-1.3.28-py3-none-any.whl (1.2 MB)
  Attempting uninstall: ibm-watsonx-ai
    Found existing installation: ibm_watsonx_ai 1.3.26
    Uninstalling ibm_watsonx_ai-1.3.26:
      Successfully uninstalled ibm_watsonx_ai-1.3.26
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [langchain_ibm]]


In [10]:
pip install pypdf

  Using cached pypdf-5.7.0-py3-none-any.whl.metadata (7.2 kB)
Using cached pypdf-5.7.0-py3-none-any.whl (305 kB)
Note: you may need to restart the kernel to use updated packages.


In [17]:
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams, EmbedTextParamsMetaNames
from ibm_watsonx_ai import Credentials
from langchain_ibm import WatsonxLLM, WatsonxEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import RetrievalQA

import os

# ----- Config -----
MODEL_ID = "mistralai/mixtral-8x7b-instruct-v01"
EMBED_MODEL_ID = "ibm/slate-125m-english-rtrvr"
PROJECT_ID = "skills-network"
WATSONX_URL = "https://us-south.ml.cloud.ibm.com"

# ----- LLM -----
def get_llm():
    parameters = {
        GenParams.MAX_NEW_TOKENS: 256,
        GenParams.TEMPERATURE: 0.5,
    }
    return WatsonxLLM(model_id=MODEL_ID, url=WATSONX_URL, project_id=PROJECT_ID, params=parameters)

# ----- Embedding Model -----
def get_embedding_model():
    embed_params = {
        EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
        EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
    }
    return WatsonxEmbeddings(model_id=EMBED_MODEL_ID, url=WATSONX_URL, project_id=PROJECT_ID, params=embed_params)


In [11]:
loader = PyPDFLoader("paper.pdf")
docs = loader.load()
print(docs[0].page_content[:1000])

A Comprehensive Review of Low-Rank
Adaptation in Large Language Models for
Efficient Parameter Tuning
September 10, 2024
Abstract
Natural Language Processing (NLP) often involves pre-training large
models on extensive datasets and then adapting them for specific tasks
through fine-tuning. However, as these models grow larger, like GPT-3
with 175 billion parameters, fully fine-tuning them becomes computa-
tionally expensive. We propose a novel method called LoRA (Low-Rank
Adaptation) that significantly reduces the overhead by freezing the orig-
inal model weights and only training small rank decomposition matrices.
This leads to up to 10,000 times fewer trainable parameters and reduces
GPU memory usage by three times. LoRA not only maintains but some-
times surpasses fine-tuning performance on models like RoBERTa, De-
BERTa, GPT-2, and GPT-3. Unlike other methods, LoRA introduces
no extra latency during inference, making it more efficient for practical
applications. All relevant code an

In [19]:
latex_text = r"""
    \documentclass{article}
    
    \begin{document}
    
    \maketitle
    
    \section{Introduction}
    Large language models (LLMs) are a type of machine learning model that can be trained on vast amounts of text data to generate human-like language. In recent years, LLMs have made significant advances in a variety of natural language processing tasks, including language translation, text generation, and sentiment analysis.
    
    \subsection{History of LLMs}
    The earliest LLMs were developed in the 1980s and 1990s, but they were limited by the amount of data that could be processed and the computational power available at the time. In the past decade, however, advances in hardware and software have made it possible to train LLMs on massive datasets, leading to significant improvements in performance.
    
    \subsection{Applications of LLMs}
    LLMs have many applications in industry, including chatbots, content creation, and virtual assistants. They can also be used in academia for research in linguistics, psychology, and computational linguistics.
    
    \end{document}
"""

latex_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.LATEX, chunk_size=60, chunk_overlap=0
)
latex_docs = latex_splitter.create_documents([latex_text])
latex_docs

[Document(metadata={}, page_content='\\documentclass{article}\n\n    \\begin{document}'),
 Document(metadata={}, page_content='\\maketitle\n\n    \\section{Introduction}\n    Large language'),
 Document(metadata={}, page_content='models (LLMs) are a type of machine learning model that can'),
 Document(metadata={}, page_content='be trained on vast amounts of text data to generate'),
 Document(metadata={}, page_content='human-like language. In recent years, LLMs have made'),
 Document(metadata={}, page_content='significant advances in a variety of natural language'),
 Document(metadata={}, page_content='processing tasks, including language translation, text'),
 Document(metadata={}, page_content='generation, and sentiment analysis.'),
 Document(metadata={}, page_content='\\subsection{History of LLMs}\n    The earliest LLMs were'),
 Document(metadata={}, page_content='developed in the 1980s and 1990s, but they were limited by'),
 Document(metadata={}, page_content='the amount of data that

In [20]:
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from langchain_ibm import WatsonxEmbeddings

In [26]:
embed_params = {
    EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
    EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
}

watsonx_embedding = WatsonxEmbeddings(
    model_id="ibm/slate-125m-english-rtrvr",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="6a014194-5845-4f2c-ada7-8e032e2de665",
    params=embed_params,
    apikey="2OiYfaZ6sM-ijKK7roYWEP1NfHjXXO8lqRlSA0UtWNoW"
)

query = "How are you?"
query_result = watsonx_embedding.embed_query(query)
query_result[:5]

[-0.06722454, -0.023729993, 0.017487843, -0.013195328, -0.039584607]

In [31]:
loader = TextLoader("new-Policies.txt")
data = loader.load()
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
)
chunks = text_splitter.split_documents(data)
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from langchain_ibm import WatsonxEmbeddings
embed_params = {
    EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
    EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
}


ids = [str(i) for i in range(0, len(chunks))]
vectordb = Chroma.from_documents(chunks, watsonx_embedding, ids=ids)

results = vectordb.similarity_search("Smoking policy", k=5)
for i, doc in enumerate(results):
    print(f"Result {i+1}: {doc.page_content[:200]}")


Result 1: This policy encourages the responsible use of mobile devices in line with legal and ethical
Result 2: This policy promotes the safe and responsible use of digital communication tools in line with our
Result 3: This policy lays the foundation for a diverse, inclusive, and talented workforce. It ensures that
Result 4: guidelines. The policy is regularly reviewed to stay current with evolving technology and security
Result 5: We encourage a culture of safety, including reporting any unsafe practices or conditions.


In [32]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})
query = "smoemailking policy"
docs = retriever.invoke(query)
docs

[Document(metadata={'source': 'new-Policies.txt'}, page_content='and email use, including copyright and data protection laws.'),
 Document(metadata={'source': 'new-Policies.txt'}, page_content='messaging apps or emails. Exercise caution when discussing company matters in public.')]